# Lab 3 - Consulting ProjectUse Case

### Numan SAHNOU & Matthieu ECCHER

In [11]:
from sklearn import datasets
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


df = pd.read_csv("loan_prediction.csv")

df = df.set_index(['Loan_ID'])
df
#df[(df['Credit_History']!=0) & (df['Credit_History']!=1)]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y


# Missing values

In [12]:
df_cp = df

df_cp.isna().any().any() #We have missing values

df_cp.isna().sum().sum() #We have 149 missing values

df_cp.loc[:, df_cp.isnull().any()].columns #We know that Gender, Married, Dependents, Self_Employed, LoanAmount, Loan_Amount_Term, Credit_History contains missing values

df_cp['Gender'].value_counts() #Male is more represented
df_cp.Gender = df_cp.Gender.fillna('Male')

df_cp['Married'].value_counts() #Yes is more represented
df_cp.Married = df_cp.Married.fillna('Yes')

df_cp['Dependents'].value_counts() #0 is more represented
df_cp.Dependents = df_cp.Dependents.fillna('0')

df_cp['Self_Employed'].value_counts() #No is more represented
df_cp.Self_Employed = df_cp.Self_Employed.fillna('No')

df_cp.LoanAmount = df_cp.LoanAmount.fillna(df_cp.LoanAmount.median())

df_cp['Loan_Amount_Term'].value_counts() #360 is more represented
df_cp.Loan_Amount_Term = df_cp.Loan_Amount_Term.fillna(360)

df_cp['Credit_History'].value_counts() #1 is more represented
df_cp = df_cp.dropna(subset=['Credit_History'])

df_cp['totalIncome'] = df_cp['ApplicantIncome'] + df_cp['CoapplicantIncome']
df_cp.loc[ df_cp['totalIncome'] <= 6000, 'totalIncome'] = 0
df_cp.loc[(df_cp['totalIncome'] > 6000) & (df_cp['totalIncome'] <= 11000), 'totalIncome'] = 1
df_cp.loc[(df_cp['totalIncome'] > 11000) & (df_cp['totalIncome'] <= 17000), 'totalIncome'] = 2
df_cp.loc[(df_cp['totalIncome'] > 17000) & (df_cp['totalIncome'] <= 33000), 'totalIncome'] = 3
df_cp.loc[(df_cp['totalIncome'] > 33000) & (df_cp['totalIncome'] <= 49000), 'totalIncome'] = 4
df_cp.loc[(df_cp['totalIncome'] > 49000) & (df_cp['totalIncome'] <= 65000), 'totalIncome'] = 5
df_cp.loc[ df_cp['totalIncome'] > 65000, 'totalIncome'] = 6

df_cp.loc[ df_cp['Loan_Amount_Term'] <= 120, 'Loan_Amount_Term'] = 0
df_cp.loc[(df_cp['Loan_Amount_Term'] > 120) & (df_cp['Loan_Amount_Term'] <= 240), 'Loan_Amount_Term'] = 1
df_cp.loc[(df_cp['Loan_Amount_Term'] > 240) & (df_cp['Loan_Amount_Term'] <= 360), 'Loan_Amount_Term'] = 2
df_cp.loc[(df_cp['Loan_Amount_Term'] > 360) & (df_cp['Loan_Amount_Term'] <= 480), 'Loan_Amount_Term'] = 3

df_cp

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/venv/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,totalIncome
Loan_ID,,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,2.0,1.0,Urban,Y,0.0
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,2.0,1.0,Rural,N,1.0
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,2.0,1.0,Urban,Y,0.0
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,2.0,1.0,Urban,Y,0.0
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,2.0,1.0,Urban,Y,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,2.0,1.0,Rural,Y,0.0
LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,1.0,1.0,Rural,Y,0.0
LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,2.0,1.0,Urban,Y,1.0


In [13]:
df_cp.Gender = df_cp.Gender.map({'Male': 1, 'Female':0})
df_cp.Loan_Status = df_cp.Loan_Status.map({'Y': 1, 'N':0})
df_cp.Education = df_cp.Education.map({'Graduate': 1, 'Not Graduate':0})
df_cp.Self_Employed = df_cp.Self_Employed.map({'No': 0, 'Yes':1})
df_cp.Married = df_cp.Married.map({'No': 0, 'Yes':1})
df_cp.Dependents = df_cp.Dependents.map({'0': 0, '1':1, '2':2, '3+':3})
df_cp.Property_Area = df_cp.Property_Area.map({'Rural': 0, 'Urban':1, 'Semiurban':2})

#df_cp['LoanAmount'] = 1000*df_cp['LoanAmount']

df_cp
#df_cp.isna().sum().sum() # We have now 79 missing values


/opt/venv/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,totalIncome
Loan_ID,,,,,,,,,,,,,
LP001002,1,0,0,1,0,5849,0.0,128.0,2.0,1.0,1,1,0.0
LP001003,1,1,1,1,0,4583,1508.0,128.0,2.0,1.0,0,0,1.0
LP001005,1,1,0,1,1,3000,0.0,66.0,2.0,1.0,1,1,0.0
LP001006,1,1,0,0,0,2583,2358.0,120.0,2.0,1.0,1,1,0.0
LP001008,1,0,0,1,0,6000,0.0,141.0,2.0,1.0,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,0,0,0,1,0,2900,0.0,71.0,2.0,1.0,0,1,0.0
LP002979,1,1,3,1,0,4106,0.0,40.0,1.0,1.0,0,1,0.0
LP002983,1,1,1,1,0,8072,240.0,253.0,2.0,1.0,1,1,1.0


# Definition of features & labels

In [14]:
X = df_cp[['Married','Dependents','Education', 'Self_Employed', 'totalIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']]
y = df_cp[['Loan_Status']]

X

,Married,Dependents,Education,Self_Employed,totalIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,
LP001002,0,0,1,0,0.0,128.0,2.0,1.0,1
LP001003,1,1,1,0,1.0,128.0,2.0,1.0,0
LP001005,1,0,1,1,0.0,66.0,2.0,1.0,1
LP001006,1,0,0,0,0.0,120.0,2.0,1.0,1
LP001008,0,0,1,0,0.0,141.0,2.0,1.0,1
...,...,...,...,...,...,...,...,...,...
LP002978,0,0,1,0,0.0,71.0,2.0,1.0,0
LP002979,1,3,1,0,0.0,40.0,1.0,1.0,0
LP002983,1,1,1,0,1.0,253.0,2.0,1.0,1


# features ingenering

In [15]:
X= pd.get_dummies(X, columns = ['Property_Area'])
X= pd.get_dummies(X, columns = ['totalIncome'])
X= pd.get_dummies(X, columns = ['Loan_Amount_Term'])
X= pd.get_dummies(X, columns = ['Dependents'])

X_train, X_test = train_test_split(X, test_size = 0.0001, random_state=0)
y_train, y_test = train_test_split(y, test_size = 0.0001, random_state=0)

X_train

,Married,Education,Self_Employed,LoanAmount,Credit_History,Property_Area_0,Property_Area_1,Property_Area_2,totalIncome_0.0,totalIncome_1.0,...,totalIncome_5.0,totalIncome_6.0,Loan_Amount_Term_0.0,Loan_Amount_Term_1.0,Loan_Amount_Term_2.0,Loan_Amount_Term_3.0,Dependents_0,Dependents_1,Dependents_2,Dependents_3
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP002126,1,0,0,74.0,1.0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,1
LP001691,1,0,0,124.0,1.0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,1,0
LP002129,1,1,0,160.0,1.0,0,0,1,1,0,...,0,0,0,0,1,0,1,0,0,0
LP002434,1,0,0,110.0,1.0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
LP001977,1,1,0,96.0,1.0,0,1,0,1,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP001963,1,1,0,113.0,1.0,0,1,0,1,0,...,0,0,0,0,1,0,0,1,0,0
LP001020,1,1,0,349.0,1.0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
LP002250,1,1,0,125.0,1.0,1,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0


# Application

In [16]:
def normalizeFeatures(df):
    df_norm = df.copy()
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm

X_train = normalizeFeatures(X_train)

X_train.insert (0, "intercept", np.nan)
X_train= X_train.fillna(1)


X_test = normalizeFeatures(X_test)
X_test.insert (0, "intercept", np.nan)
X_test= X_test.fillna(1)

X_train_numpy = X_train.to_numpy()
y_train_numpy = y_train.to_numpy()

X_test_numpy = X_test.to_numpy()
y_test_numpy = y_test.to_numpy()

m = np.size(X_train,0)
n = np.size(X_train,1)

X_train

,intercept,Married,Education,Self_Employed,LoanAmount,Credit_History,Property_Area_0,Property_Area_1,Property_Area_2,totalIncome_0.0,...,totalIncome_5.0,totalIncome_6.0,Loan_Amount_Term_0.0,Loan_Amount_Term_1.0,Loan_Amount_Term_2.0,Loan_Amount_Term_3.0,Dependents_0,Dependents_1,Dependents_2,Dependents_3
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP002126,1.0,1.0,0.0,0.0,0.094067,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
LP001691,1.0,1.0,0.0,0.0,0.166425,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
LP002129,1.0,1.0,1.0,0.0,0.218524,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
LP002434,1.0,1.0,0.0,0.0,0.146165,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
LP001977,1.0,1.0,1.0,0.0,0.125904,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP001963,1.0,1.0,1.0,0.0,0.150507,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
LP001020,1.0,1.0,1.0,0.0,0.492041,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
LP002250,1.0,1.0,1.0,0.0,0.167873,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


# Creation of weights

In [17]:
import random
def weight(df):
    w = []
    for i in range (0, len(df.axes[1])):
        w.append(random.uniform(-10, 10)) 
    np.array(w)
    
    return np.transpose([w])

w = weight(X_train)

# Sigmoid Function

In [18]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

# Cost Function

In [19]:
def costFunction(h, y):    
    return -(1/m) * np.sum( (y*np.log(h)) + ((1-y)*np.log(1-h)))

# Gradient Descent

In [20]:
def gradient_descent(X, y, w, learning_rate, iterations, d):

    cost_history = []
    w_history = np.zeros((iterations,n))
    
    for i in range(iterations):
        z = np.dot(X, w)
        h = sigmoid(z)
        w = w - learning_rate * X.T.dot((h-y))
        w_history[i,:] = w.T
        cost_history.append(costFunction(h, y))

        print('iteration : ', i, ' | cout : ', cost_history[i], ' | difference pred-actual : ', cost_history[i-1]-cost_history[i], ' | delta condition : ', np.absolute(cost_history[i-1]-cost_history[i]) < d, ' | alpha : ', learning_rate)
        
        if (cost_history[i-1] - cost_history[i] < 0):
            learning_rate = learning_rate / 2
        elif(i > 0 and np.absolute(cost_history[i-1] - cost_history[i]) < d):
            return w, cost_history, w_history

    return w, cost_history, w_history

In [21]:
alpha = 0.001
iterations = 100000
delta = 0.000001
cost_history = []
w, cost_history, w_history = gradient_descent(X_train_numpy, y_train_numpy , w, alpha, iterations, delta)

print(w.T)
print('Final cost :', cost_history[-1])


iteration :  2779  | cout :  0.43736625556622477  | difference pred-actual :  2.833564747284356e-06  | delta condition :  False  | alpha :  0.001
iteration :  2780  | cout :  0.4373634247896365  | difference pred-actual :  2.8307765882473745e-06  | delta condition :  False  | alpha :  0.001
iteration :  2781  | cout :  0.4373605967990126  | difference pred-actual :  2.827990623899268e-06  | delta condition :  False  | alpha :  0.001
iteration :  2782  | cout :  0.43735777159215955  | difference pred-actual :  2.8252068530743024e-06  | delta condition :  False  | alpha :  0.001
iteration :  2783  | cout :  0.4373549491668846  | difference pred-actual :  2.8224252749398104e-06  | delta condition :  False  | alpha :  0.001
iteration :  2784  | cout :  0.4373521295209961  | difference pred-actual :  2.8196458884965914e-06  | delta condition :  False  | alpha :  0.001
iteration :  2785  | cout :  0.43734931265230326  | difference pred-actual :  2.816868692856467e-06  | delta condition :  Fa

In [22]:
y_predicted = sigmoid(w.T.dot(X_train_numpy.T).T)

y_predicted[y_predicted > 0.5] = 1
y_predicted[y_predicted <= 0.5] = 0

y_predicted = y_predicted.astype(int)
y_actual = y_train.to_numpy()
y_predicted

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
    

# Score

In [36]:
def score_from_scratch(y_predicted, y_actual):
    cpt = 0
    truepositive=0
    truenegative=0
    falsepositive=0
    falsenegative=0
    for i in range(len(y_predicted)):
        if(y_predicted[i] == y_actual[i] == 1 ):
            cpt = cpt+1
            truepositive=truepositive+1
        elif(y_predicted[i] == y_actual[i] == 0 ):
            cpt = cpt+1
            truenegative= truenegative+1
        elif(y_predicted[i] != y_actual[i]):
            if(y_predicted[i]==1):
                falsepositive= falsepositive+1
            if(y_predicted[i]==0):        
                falsenegative= falsenegative+1
            
    
    return cpt/len(y_predicted)

score_from_scratch(y_predicted, y_actual),f1_score(y_actual,y_predicted)

(0.8259325044404974, 0.8849765258215962)

# Matrix from scratch

In [41]:
def matrix_from_scratch(y_predicted, y_actual):
    truepositive=0
    truenegative=0
    falsepositive=0
    falsenegative=0
    for i in range(len(y_predicted)):
        if(y_predicted[i] == y_actual[i] == 1 ):
            truepositive=truepositive+1
        elif(y_predicted[i] == y_actual[i] == 0 ):
            truenegative= truenegative+1
        elif(y_predicted[i] != y_actual[i]):
            if(y_predicted[i]==1):
                falsepositive= falsepositive+1
            if(y_predicted[i]==0):        
                falsenegative= falsenegative+1
            
    tab= np.zeros((2,2)) 
    tab[1][1] = int(truepositive)
    tab[0][1] = int(falsepositive)
    tab[0][0] = int(truenegative)
    tab[1][0] = int(falsenegative) 
    return tab

matrix_from_scratch(y_predicted, y_actual)

array([[ 88.,  90.],
       [  8., 377.]])

# Scikit Learn comparison

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

model = LogisticRegression(verbose=1)
model.fit(X_train_numpy,y_train)
y_pred = model.predict(X_train)


model.intercept_,model.coef_, model.score(X_train,y_train), f1_score(y_actual,y_pred)

/opt/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


(array([-3.19741966]),
 array([[ 7.05818615e-05,  4.62912210e-01,  3.93137663e-01,
         -4.76999287e-02, -3.43809444e-01,  3.49693230e+00,
         -3.51745668e-01, -2.31872695e-01,  5.83688945e-01,
          2.11532750e-01,  6.49676292e-02,  2.91073442e-01,
         -5.99097814e-01, -2.44560826e-01,  3.59573302e-01,
         -8.34179029e-02, -1.31493972e-01,  5.59774081e-01,
          4.38446014e-01, -8.66655541e-01, -5.24672533e-02,
         -2.44289642e-01,  1.83611449e-01,  1.13216028e-01]]),
 0.8206039076376554,
 0.8815943728018757)

# Confusion Matrix & comparison

In [25]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_pred)
cm_fromscratch = confusion_matrix(y_train, y_predicted)

#Thus in binary classification, the count of true negatives is C (0,0) , false negatives is C (1,0) , true positives is C (1,1)  and false positives is C (0,1) .

print("Confusion matrix comparison\n\n")
print("- true negatives \n \tScikit Learn :",cm[0][0],'| From Scratch :',cm_fromscratch[0][0])
print("- false negatives \n \tScikit Learn : ",cm[1][0],'| From Scratch :',cm_fromscratch[1][0])
print("- true positives \n \tScikit Learn :",cm[1][1],'| From Scratch :',cm_fromscratch[1][1])
print("- false positives \n \tScikit Learn :",cm[0][1],'| From Scratch :',cm_fromscratch[0][1])


Confusion matrix comparison


- true negatives 
 	Scikit Learn : 86 | From Scratch : 88
- false negatives 
 	Scikit Learn :  9 | From Scratch : 8
- true positives 
 	Scikit Learn : 376 | From Scratch : 377
- false positives 
 	Scikit Learn : 92 | From Scratch : 90


# Option

## Random forest

In [26]:
from sklearn.ensemble import RandomForestClassifier

model_ensemble = RandomForestClassifier(max_depth = 10, random_state = 0)
model_ensemble.fit(X_train, y_train)
y_pred2 = model_ensemble.predict(X_train)

print(model_ensemble.score(X_train, y_train))
confusion_matrix(y_train, y_pred2)

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
0.8969804618117229


array([[120,  58],
       [  0, 385]])